In [1]:
from numpy import array
from numpy import asarray
from numpy import zeros
import string
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Embedding
import pickle
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
import IPython; from IPython.display import display, HTML
def dfPrint(df):
    display(HTML(df.to_html()))

[nltk_data] Downloading package stopwords to /home/atharv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Using TensorFlow backend.


In [2]:
def import_data():
    train_path = "./data/nlp_trip_advisor/train.csv"
    data = pd.read_csv(train_path)
    data = data[data.Is_Response.isnull() == False]
#     data['Is_Response'] = data['Is_Response'].map(int)
    data = data[data['Description'].isnull() == False]
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)
    print ('dataset loaded with shape', data.shape)
    data = pd.get_dummies(data, columns=["Is_Response"])
    data.drop(['Browser_Used', 'Device_Used', 'Is_Response_not happy'], axis=1, inplace=True)
    data.columns = ['User_ID', "Description", "Sentiment"]
    return data

data = import_data()
data = data.iloc[:]
data.to_csv("./data/data_train.csv")
dfPrint(data.sample(5))
# data['Sentiment'].value_counts()
data_sent_1 = data[data["Sentiment"]==1]
data_sent_1.to_csv("./data/data_sent_1_train.csv")
dfPrint(data_sent_1.sample(5))
data_sent_0 = data[data["Sentiment"]==0]
data_sent_0.to_csv("./data/data_sent_0_train.csv")
dfPrint(data_sent_0.sample(5))

dataset loaded with shape (38932, 5)


,User_ID,Description,Sentiment
18929,id29255,I don't know what my firm paid for this one. B...,0
36974,id47300,I'm not sure where the SLS brand is trying to ...,1
10601,id20927,My wife and I stayed at the U.S Grant mid augu...,1
13403,id23729,Quaint hotel. Nice and clean room. Room amenit...,1
23116,id33442,We spent a week at the River Inn and thoroughl...,1


,User_ID,Description,Sentiment
24679,id35005,The view of marine bay and the beach is nice a...,1
3436,id13762,We stayed here for a short trip to Chicago. Th...,1
9227,id19553,We just stayed at this hotel during a one week...,1
12000,id22326,Excellent choice for business traveler or tour...,1
7246,id17572,We stayed over the Independence weekend for a ...,1


,User_ID,Description,Sentiment
5278,id15604,We were in town for a conference and were put ...,0
2014,id12340,After staying in Vegas and Anaheim we were loo...,0
9948,id20274,Overall our stay was nice here. The hotel is o...,0
7667,id17993,I have stayed in many hotels while traveling t...,0
10365,id20691,I want to preface this review by pointing out ...,0


In [3]:
def clean_decription(line):
    words = list(set(line.split()))
    words = list(sorted([x.lower() for x in words]))
#     print (len(words))
#     print (words)
    words2 = words.copy()
    for word in list(set(words2).intersection(stop_words)):
        words.remove(word)
#     out = ' '.join(words)
    out = ' '.join(e for e in words if e.isalnum())
    if out == "":
        print(words)
#     table = str.maketrans({key: None for key in string.punctuation})
#     out.translate(table, string.punctuation)
    return out
data["Description"] = data["Description"].map(clean_decription)
#     print (len(words))
#     print (words)
#     break
# data

In [16]:
docs = data["Description"]; labels = data["Sentiment"]
print(len (data["Sentiment"]))
print(len (labels))
print(len (data["Description"]))
print(len(docs))
docs_sent_1 = data_sent_1["Description"]; labels_sent_1 = data_sent_1["Sentiment"]
docs_sent_0 = data_sent_0["Description"]; labels_sent_0 = data_sent_0["Sentiment"]

38932
38932
38932
38932


In [17]:
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
print(vocab_size)

39586


In [18]:
print(len(t.word_counts))
# print(t.word_counts)
# wordCounts = sorted(t.word_counts.iteritems(), key=lambda k: (v,k))
wordCounts = sorted(t.word_counts.items(), key=lambda x:x[1], reverse=True)
print(len(wordCounts))
# print(wordCounts.items()[0])

39585
39585


In [19]:
encoded_docs = t.texts_to_sequences(docs)

In [20]:
# tf_idf = t.texts_to_matrix(docs, mode='count')

# fileObject = open("./data/PickledForKhadtare",'r')  
# # load the object from the file into var b
# # fileObject.encode('utf-8').strip()
# tf_idf = pickle.load(pybytes(readbytes(fileObject)))
# fileObject.close()

# print(encoded_docs[0])

In [21]:
# print (tf_idf)

In [22]:
# pad documents to a max length of 4 words
max_length = 250
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(len(padded_docs))
print(padded_docs)
# load the whole embedding into memory

38932
[[ 461   99   15 ...    0    0    0]
 [ 891 1442   52 ...    0    0    0]
 [  22   22  366 ...    0    0    0]
 ...
 [1052   22  100 ...    0    0    0]
 [1010   99 4366 ...    0    0    0]
 [  87 6446 7876 ...    0    0    0]]


In [23]:
embeddings_index = dict()
f = open('./data/glove/glove.6B.100d.txt')
vocab_size = 0
maxLen = 0
for line in f:
    values = line.split()
#     print (word, end = '\t')
    word = values[0]
    maxLen = max(maxLen, len(word))
    if word in stop_words:
#         print (word)
        continue
    vocab_size += 1
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# print(maxLen)

Loaded 399851 word vectors.


In [24]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
mil = 0
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        mil+=1
#         print (word, end = '\t')
print(mil)

29657


In [ ]:
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=True)
model.add(e)
model.add(Flatten())
model.add(Dense(75, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.3))
# model.add(Dense(50, kernel_initializer='normal', activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(25, kernel_initializer='normal', activation='relu'))
# model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())
print (len(labels))
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=1, validation_split=0.2)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=1)
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 250, 100)          39985100  
_________________________________________________________________
flatten_5 (Flatten)          (None, 25000)             0         
_________________________________________________________________
dense_17 (Dense)             (None, 75)                1875075   
_________________________________________________________________
dropout_13 (Dropout)         (None, 75)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 76        
Total params: 41,860,251
Trainable params: 41,860,251
Non-trainable params: 0
_________________________________________________________________
None
38932
Train on 31145 samples, validate on 7787 samples
Epoch 1/50
21248/31145 [===================>..........] - ETA: 3:50 - 